<div style="max-width:1200px"><img src="../_resources/mgnify_banner.png" width="100%"></div>

<img src="../_resources/mgnify_logo.png" width="200px">

# Mapping samples from the [AtlantECO Super Study](https://www.ebi.ac.uk/metagenomics/super-studies/atlanteco)
### ... using the MGnify API and an interactive map widget

The [MGnify API](https://www.ebi.ac.uk/metagenomics/api/v1) returns JSON data. The `jsonapi_client` package can help you load this data into Python, e.g. into a Pandas dataframe.

**This example shows you how to load a MGnify AtlantECO Super Study's data from the MGnify API and display it on an interactive world map**

You can find all of the other "API endpoints" using the [Browsable API interface in your web browser](https://www.ebi.ac.uk/metagenomics/api/v1). The URL you see in the browsable API is exactly the same as the one you can use in this code.

This is an interactive code notebook (a Jupyter Notebook).
To run this code, click into each cell and press the ▶ button in the top toolbar, or press `shift+enter`.

**Content:**
- Fetch AtlantECO studies data
- Show study' samples on the interactive world map
- Check functional annotation terms presence/absense
    - GO-term
    - InterPro 
    - Biosynthetic Gene Clusters (BGC)
    

---

## Fetch all [AtlantECO](https://www.ebi.ac.uk/metagenomics/super-studies/atlanteco) studies
A Super Study is a collection of MGnify Studies originating from a major project. AtlantECO is one such project, aiming to develop and apply a novel, unifying framework that provides knowledge-based resources for a better understanding and management of the Atlantic Ocean and its ecosystem services.

Fetch the Super Study's Studies from the MGnify API, into a [Pandas dataframe](https://pandas.pydata.org/docs/user_guide/index.html):

In [ ]:
import pandas as pd
from jsonapi_client import Session, Modifier

atlanteco_endpoint = 'super-studies/atlanteco/flagship-studies'
with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    studies = map(lambda r: r.json, mgnify.iterate(atlanteco_endpoint))
    studies = pd.json_normalize(studies)
studies[:5]

## Show the studies' samples on a map

We can fetch the Samples for each Study, and concatenate them all into one Dataframe.
Each sample has geolocation data in its `attributes` - this is what we need to build a map.

It takes time to fetch data for all samples, so **let's show samples from chosen PRJEB46727 study only.** This study contains assembly data https://www.ebi.ac.uk/metagenomics/studies/MGYS00005810#overview.

In [ ]:
substudy = studies[studies['attributes.bioproject'] == 'PRJEB46727']
studies_samples = []

with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    for idx, study in substudy.iterrows():
        print(f"fetching {study.id} samples")
        samples = map(lambda r: r.json, mgnify.iterate(f'studies/{study.id}/samples?page_size=1000'))
        samples = pd.json_normalize(samples)
        samples = pd.DataFrame(data={
            'accession': samples['id'],
            'sample_id': samples['id'],
            'study': study.id, 
            'lon': samples['attributes.longitude'],
            'lat': samples['attributes.latitude'],
            'color': "#FF0000",
        })
        samples.set_index('accession', inplace=True)
        studies_samples.append(samples)
studies_samples = pd.concat(studies_samples)

In [ ]:
print(f"fetched {len(studies_samples)} samples")

studies_samples.head()

In [ ]:
import leafmap
m = leafmap.Map(center=(0, 0), zoom=2)
m.add_points_from_xy(
    studies_samples,
    x='lon', 
    y='lat', 
    popup=["study", "sample_id"], 
    color_column='color',
    add_legend=False
)
m

## Check functional annotation terms presence
Let's check whether a specific identifier is present in each sample. 

We will work with MGnify analyses (`MGYA`s) corresponding to chosen samples. We filter analyses by 
- pipeline version: 5.0
- experiment type: assembly

This example shows how to process **just the first 10 samples** (again, because the full dataset takes a while to fetch).
Firstly, get analyses for each sample.

In [ ]:
analyses = []
with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    for idx, sample in studies_samples[:10].iterrows():
        print(f"processing {sample.sample_id}")
        filtering = Modifier(f"pipeline_version=5.0&sample_accession={sample.sample_id}&experiment_type=assembly")
        analysis = map(lambda r: r.json, mgnify.iterate('analyses', filter=filtering))
        analysis = pd.json_normalize(analysis)
        analyses.append(analysis)
analyses = pd.concat(analyses)
analyses[:5]

Define functions:
- `identify_existance` to check each analysis for identifier presence/absence. We add a column to the dataframe with a colour: blue if identifier was found and red if not.
- `show_on_map` to plot results on the world map. Join the analyses and sample tables to have geolocation data and identifier presence data together (we'll create a new sub-DataFrame with a subset of the fields and add them to the map).

In [ ]:
def identify_existance(input_analyses, identifier, term):
    data = []
    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
        for idx, mgya in input_analyses.iterrows():
            print(f"processing {mgya.id}")
            analysis_identifier = map(lambda r: r.json, mgnify.iterate(f'analyses/{mgya.id}/{identifier}'))
            analysis_identifier = pd.json_normalize(analysis_identifier)
            data.append("#0000FF" if term in list(analysis_identifier.id) else "#FF0000")
        presented = sum([1 for i in data if i == "#0000FF"])
        print(f"Presented {presented} of {identifier} {term}")
        input_analyses.insert(2, identifier, data, True)
    return input_analyses

def show_on_map(input_analyses, studies_samples, identifier):
    df = input_analyses.join(studies_samples.set_index('sample_id'), on='relationships.sample.data.id')
    df2 = df[[identifier, 'lon', 'lat', 'study', 'attributes.accession', 'relationships.study.data.id', 'relationships.sample.data.id', 'relationships.assembly.data.id']].copy()
    df2 = df2.set_index("study")
    df2 = df2.rename(columns={"attributes.accession": "analysis_ID", 
                              'relationships.study.data.id': "study_ID",
                              'relationships.sample.data.id': "sample_ID", 
                              'relationships.assembly.data.id': "assembly_ID"
                             })
    m = leafmap.Map(center=(0, 0), zoom=2)
    m.add_points_from_xy(df2, 
                         x='lon', 
                         y='lat', 
                         popup=["study_ID", "sample_ID", "assembly_ID", "analysis_ID", identifier],
                        color_column=identifier, add_legend=False)
    return m

## GO term
This example is written for GO-term for **biotin transport** [GO:0015878](http://www.candidagenome.org/cgi-bin/GO/go.pl?goid=15878)

Other GO identifiers are available on the MGnify API.

In [ ]:
identifier = "go-terms"
go_term = 'GO:0015878'
go_analyses = analyses
go_data = identify_existance(go_analyses, identifier, go_term)
map_vis = show_on_map(go_data, studies_samples, identifier)
map_vis

## InterPro entry
This example is written for InterPro entry [IPR001650](https://www.ebi.ac.uk/interpro/entry/InterPro/IPR001650): **Helicase, C-terminal domain-like**

Other IPS identifiers are available on the MGnify API.

In [ ]:
identifier = "interpro-identifiers"
ips_term = 'IPR001650'
ips_analyses = analyses
ips_data = identify_existance(ips_analyses, identifier, ips_term)
map_vis = show_on_map(ips_data, studies_samples, identifier)
map_vis

## BGC (Biosynthetic Gene Clusters)

MGnify has additional analysis of [BGCs](https://mibig.secondarymetabolites.org/) provided by [Sanntis](https://github.com/Finn-Lab/SanntiS). These annotations are saved as [RO-Crates](https://www.researchobject.org/ro-crate/) objects and linked to assembly records.

The following example counts the number of **truncated from beggining** proteins of nearest MiBIG class **Polyketide** with dice distance more than **0.65**. We will use [gffutils](https://daler.github.io/gffutils/index.html) for parsing GFF file.


Define a function to find GFF file in zipped archive by url:

In [ ]:
import requests
from zipfile import ZipFile
from io import BytesIO

def find_gff_file(link):
    # Read archive and find GFF file
    
    response = requests.get(link)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Open the zip file from the content of the response
        with ZipFile(BytesIO(response.content)) as zip_file:
            # List all files in the zip archive
            file_list = zip_file.namelist()

            # Filter files with .gff extension
            gff_files = [file_name for file_name in file_list if file_name.endswith(".gff")]
            print(f"Found {gff_files}")
            # Read the first .gff file (you can modify this to read a specific file)
            if gff_files:
                first_gff_file = gff_files[0]
                gff_content = zip_file.open(first_gff_file).read()
                return gff_content
            else:
                print("No .gff files found in the zip archive.")
                return None
    else:
        print("Failed to fetch the zip file.")
        return None

Define a function to get counts from GFF.

In [ ]:
import gffutils

def get_count(nearest_MiBIG_class, nearest_MiBIG_diceDistance, partial_value, gff_content):
    if gff_content:
        try:
            with tempfile.NamedTemporaryFile(delete=False) as temp_gff_file:
                temp_gff_file.write(gff_content)
                temp_gff_file_path = temp_gff_file.name

                # Create a GFF database using gffutils
                db = gffutils.create_db(
                    temp_gff_file_path,
                    dbfn=':memory:',  # Use an in-memory database
                    force=True,       # Overwrite if the database already exists
                    keep_order=True,  # Preserve feature order 
                )

                count = 0
                for feature in db.all_features():
                    if feature["nearest_MiBIG_class"][0] == nearest_MiBIG_class and \
                       float(feature["nearest_MiBIG_diceDistance"][0]) >= nearest_MiBIG_diceDistance and \
                       feature["partial"][0] == partial_value:
                        count += 1
                print(f"Count is {count}")
                return count
        except:
            print('Error in GFF DB')
            return 0
    else:
        return 0

Process data:

In [ ]:
nearest_MiBIG_class = "Polyketide" 
nearest_MiBIG_diceDistance = 0.65
partial_value = "10"

counts = []

with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    for idx, mgya in analyses.iterrows():
        # get ERZ assembly accession
        assembly = mgya['relationships.assembly.data.id']
        annotations_for_assembly = mgnify.iterate(f'assemblies/{assembly}/extra-annotations')
        sanntis_annotation = None
        for item in annotations_for_assembly:
            if 'sanntis' in item.id:
                sanntis_annotation = item.links.self
                break
        if not sanntis_annotation:
            print('Sanntis annotation was not found')
            continue
        
        print(f"processing {mgya.id} {assembly}")
        
        gff_content = find_gff_file(sanntis_annotation)
        counts.append(get_count(nearest_MiBIG_class, nearest_MiBIG_diceDistance, partial_value, gff_content))

Display on the interactive map

In [ ]:
identifier = "bgc"
analyses.insert(2, identifier, counts, True)
map_vis = show_on_map(analyses, studies_samples, identifier)
map_vis